In [1]:
import numpy as np

## Pressure Vessel Optimisation ##

# There are 4 variables that describe a pressure vessel:
# z1 = 0 # Shell thickness
# z2 = 0 # Head thickness
# x3 = 0 # Inner radius
# x4 = 0 # vessel length (excluding head)

# Our input variables:
# x1,x2 = 0,0

# Decision Vector:
# np.array(x1,x2,x3,x4)

# Use static penalty



# The shell head and thickness can be expressed as
# z1 = 0.0625* x1
# z2 = 0.0625* x2

# Counter variables:
f = 0
g1_call_count = 0
g2_call_count = 0
g3_call_count = 0
g4_call_count = 0

# Constraints:
# x1, x2 are integers between 1 and 99 
# x3, x4 are continuous values between 10 and 200

# Best known feasible solution of f(x*) = 5.8853327736×10^3


In [2]:
def calculate_zs(xs):
    z1 = (xs[0]*0.0625)
    z2 = (xs[1]*0.0625)
    return z1, z2


In [3]:
def f(xs, z1, z2):
    # f_call_count += 1
    x3 = xs[2]
    x4 = xs[3]

    xnew = (1.7781)*(z2)*(x3**2) + (0.6224)*(z1)*(x3)*(x4) + (3.1661)*(z1**2)*(x4) + (19.84)*(z1**2)*(x3)
    return xnew


In [4]:
def g1(xs, z2):
    # g1_call_count += 1
    return xs[2]*0.00954
    

def g2(xs, z1):
    # g2_call_count += 1
    return xs[2]*0.0193
    
        
def g3(xs):
    # g3_call_count += 1
    return xs[3]
  

def g4(xs):
    # g4_call_count += 1
    return (-1)*(np.pi)*(xs[2]**2)*(xs[3]) - ((4/3)*np.pi)*(xs[2]**3)


In [5]:
import numpy as np
def main():
    xs = np.array([1, 35, 11.678, 88.76])
    z1,z2 = calculate_zs(xs)
    print("Objective function output: ", f(xs, z1, z2))
    print("Constraint function 1 output: ", g1(xs, z2))
    print("Constraint function 2 output: ", g2(xs, z1))
    print("Constraint function 3 output: ", g3(xs))
    print("Constraint function 4 output: ", g4(xs))
    
    # if (g1(xs, z2) == True and g2(xs, z1) == True and g3(xs) == True and g4(xs) == True):
    #     f(xs, z1, z2)
    # else:
    #     print(f"Constraints not met!")

In [6]:
main()

Objective function output:  572.77017736475
Constraint function 1 output:  0.11140812000000001
Constraint function 2 output:  0.22538540000000004
Constraint function 3 output:  88.76
Constraint function 4 output:  -44699.10187026799


In [12]:
# RANDOM SEARCH
# [x1,x2](ints 1 <= x <= 99) [x3,x4] (floats 10 <= x <= 200)

N = 400000
np.random.seed(372)

x1_best = np.random.randint(low=1, high=99)
x2_best = np.random.randint(low=1, high=99)

x3_best = np.random.uniform(low=10, high=200)
x4_best = np.random.uniform(low=10, high=200)

best_xs = np.array([x1_best, x2_best, x3_best, x4_best])
z1, z2 = calculate_zs(best_xs)
best_output = f(best_xs, z1, z2)
this_output = 0

results = np.array([])


#z1,z2 = calculate_zs(xs)


for n in range(N):
    this_x1 = np.random.randint(low=1, high=99)
    this_x2 = np.random.randint(low=1, high=99)

    this_x3 = np.random.uniform(low=10, high=200)
    this_x4 = np.random.uniform(low=10, high=200)

    this_xs = np.array([this_x1, this_x2, this_x3, this_x4])

    this_z1, this_z2 = calculate_zs(this_xs)
    this_output = f(this_xs, this_z1, this_z2)

    if (abs(5885.3327736 - this_output) < abs(5885.3327736 - best_output)):
        x1_best = this_x1
        x2_best = this_x2
        x3_best = this_x3
        x4_best = this_x4
        best_output = this_output

        print(f'New best! f(x) = {np.format_float_scientific(this_output)}')


    



# print(x1_best,x2_best,x3_best ,x4_best)

New best! f(x) = 1.2234847377459348e+05
New best! f(x) = 8.702893805950858e+04
New best! f(x) = 3.708300656882236e+04
New best! f(x) = 3.47685350346202e+04
New best! f(x) = 1.2317817887265195e+04
New best! f(x) = 1.118378834542561e+04
New best! f(x) = 9.761558936809617e+03
New best! f(x) = 4.824105209110673e+03
New best! f(x) = 6.388618271215959e+03
New best! f(x) = 6.207455805449975e+03
New best! f(x) = 5.892459290097843e+03
New best! f(x) = 5.886764635742163e+03
New best! f(x) = 5.885243033734538e+03
New best! f(x) = 5.885279559549594e+03
